In [ ]:
import dash
import jupyter_dash
import dash_leaflet as dl
from dash import Dash, callback, html, dcc, Input, Output, State, MATCH, ALL
from dash import dash_table as dt
import plotly.express as px

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps
import base64

import animalShelter


# import AnimalShelter 
from animalShelter import AnimalShelter

# mongo login credentials
username = "aacuser1"
password = "password"
animals = AnimalShelter(username, password)

# class read method
df = pd.DataFrame.from_records(animals.read({})).drop('_id', axis = 1)
df = df.sort_values(by=['1'])
rescue_types = ['Water Rescue', 'Mountain or Wilderness Rescue', 'Disaster or Individual Tracking', 'Reset']

#########################
# Dashboard
#########################
app = JupyterDash('SimpleExample')

image_filename = 'GSLogo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app.layout = html.Div([
    # title and logo
    html.Center(children=[
        html.B(html.H1('Project 2 Dashboard')),
        html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()), style={'width': '200px'}),
        # identifier
        html.Center(html.H4('CS340 - Brady Goodwin')),
    ]),
    html.Hr(),
    
    # radio buttons for filtering animal types
    dcc.RadioItems(
                id='rescue-type-filter',
                options=[{'label': i, 'value': i} for i in rescue_types],
                value='Reset',
                labelStyle={'display': 'inline-block'}
            ),
    html.Hr(),
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),

        # data table features for client interaction
        editable=False,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable=False,
        row_selectable=False,
        row_deletable=False,
        selected_columns=[],
        selected_rows=[],
        page_action="native",
        page_current=0,
        page_size=10,
    ),
    html.Br(),
    html.Hr(),
    
    # pie chart and geolocation map
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ]),
    html.Br(),
    
    # histogram
    html.Div(
        id='second-graph-id',
        className='col s12 m6',
        ),
])

#############################################
# Interaction Between Components / Controller
#############################################

# callback for radio butons
@app.callback(Output('datatable-id','data'),
              [Input('rescue-type-filter', 'value')])
def update_dashboard(filter_type):
    if filter_type == 'Reset':
        dff = df
    elif filter_type == 'Water Rescue':
        dff = df[df.breed.isin(['Labrador Retriever Mix', 'Chesapeake Bay Retriever', 'Newfoundland'])
                & (df.sex_upon_outcome == 'Intact Female')
                & ((df.age_upon_outcome_in_weeks >= 26) & (df.age_upon_outcome_in_weeks <= 156))]
    elif filter_type == 'Mountain or Wilderness Rescue':
        dff = df[df.breed.isin(['German Shepherd', 'Alaskan Malamute', 'Old English Sheepdog', 'Siberian Husky', 'Rottweiler'])
                & (df.sex_upon_outcome == 'Intact Male')
                & ((df.age_upon_outcome_in_weeks >= 26) & (df.age_upon_outcome_in_weeks <= 156))]
    elif filter_type == 'Disaster or Individual Tracking':
        dff = df[df.breed.isin(['Doberman Pinscher', 'German Shepherd', 'Golden Retriever', 'Bloodhound', 'Rottweiler'])
                & (df.sex_upon_outcome == 'Intact Male')
                & ((df.age_upon_outcome_in_weeks >= 20) & (df.age_upon_outcome_in_weeks <= 300))]
        
    return dff.to_dict('records')

@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

# callback for pie chart
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
    dff = pd.DataFrame.from_dict(viewData)
    df_grouped = dff[['breed', '1']].groupby(['breed']).agg('count').reset_index()
    df_grouped = df_grouped.rename(columns = {'1': 'Count'})
    
    fig = px.pie(df_grouped, values='Count', names='breed', title="Animal Types")
    return [
        dcc.Graph(figure=fig)
    ]

# callback for geolocation map
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_map(viewData):
    dff = pd.DataFrame.from_dict(viewData)
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id")] +
            # Marker with tool tip and popup
            [dl.Marker(position=[row['location_lat'],row['location_long']], children=[
                dl.Tooltip(row['breed']),
                dl.Popup([
                     html.H1("Animal Name"),
                     html.P(row['name'])
                ])
            ]) for index, row in dff.iterrows()]
        )
    ]

# callback for histogram
@app.callback(
    Output('second-graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs2(viewData):
    dff = pd.DataFrame.from_dict(viewData)
    fig = px.histogram(dff, x="age_upon_outcome_in_weeks")
    return [
        dcc.Graph(figure=fig)
    ]

app